In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import scanpy as sc
from mofapy2.run.entry_point import mofa
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from mofapy2.run.entry_point import mofa
import seaborn as sns
import mofax as mfx

In [ ]:
# load sc data from infercnv
# load sn data from outlier nb because inferCNV failed for some reason, sn anyway has malignant labels
adata_sc = sc.read_h5ad('../single_cell_int/adata_sc_int_cnv.h5ad')
adata_sn = sc.read_h5ad('../single_nuc_int/adata_nuc_int_outlier_genes.h5ad')

# done with 10_MOFA.py

# 6082 HVG and 15 latent factors, Filtered

In [ ]:
m_more = mfx.mofa_model("/home/aih/shrey.parikh/PDAC/PDAC_Final/MOFA/MOFA_results_even_more_hvg_more_factors_filtered_job.hdf5")

In [ ]:
print(f"""\
Cells: {m_more.shape[0]}
Features: {m_more.shape[1]}
Groups of cells: {', '.join(m_more.groups)}
Views: {', '.join(m_more.views)}
""")

In [ ]:
dict_df = {}
for i in list(range(0,15)):
    i= i+1
    # print(i)
    temp_df = m_more.get_weights(factors=str(i), df=True).sort_values(by=str(i), ascending=False).index.tolist()
    name = 'Factor' + str(i)
    dict_df[name] = temp_df

In [ ]:
df_factor_genes = pd.DataFrame(dict_df)

In [ ]:
len(set(df_factor_genes[:400].values.flatten()))

In [ ]:
df_factor_genes.to_csv('../MOFA/MOFA_15_Factors_6082HVG/genes.csv')

In [ ]:
df_factor_genes.head()

In [ ]:
def overlap_count(col1, col2):
    return len(set(col1) & set(col2))

df = df_factor_genes[:400]
overlap_matrix = pd.DataFrame(index=df.columns, columns=df.columns)

for col1 in df.columns:
    for col2 in df.columns:
        overlap_matrix.loc[col1, col2] = overlap_count(df[col1], df[col2])

# Convert the values to integer type
overlap_matrix = overlap_matrix.astype(int)

In [ ]:
total_genes = df.shape[0]
percentage_overlap_matrix = (overlap_matrix / total_genes) * 100
percentage_overlap_matrix = percentage_overlap_matrix.astype(int)

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(percentage_overlap_matrix, annot=True, cmap="Grays", fmt='d', )
plt.title("Heatmap of Percentage Overlapping Genes between Latent Factors")
plt.yticks(rotation=0)
plt.show()

## check for first 10 factors

In [ ]:
df.shape

In [ ]:
def overlap_count(col1, col2):
    return len(set(col1) & set(col2))

df = df[:200].iloc[:, :10]
overlap_matrix = pd.DataFrame(index=df.columns, columns=df.columns)

for col1 in df.columns:
    for col2 in df.columns:
        overlap_matrix.loc[col1, col2] = overlap_count(df[col1], df[col2])

# Convert the values to integer type
overlap_matrix = overlap_matrix.astype(int)
total_genes = df.shape[0]
percentage_overlap_matrix = (overlap_matrix / total_genes) * 100
percentage_overlap_matrix = percentage_overlap_matrix.astype(int)
plt.figure(figsize=(10, 8))
sns.heatmap(percentage_overlap_matrix, annot=True, cmap="Grays", fmt='d', )
plt.title("Heatmap of Percentage Overlapping Genes between Latent Factors")
plt.yticks(rotation=0)
plt.show()

In [ ]:
latent_factors = m_more.get_factors(df=True)
latent_factors.head()

In [ ]:
latent_factors.to_csv('../MOFA/MOFA_15_Factors_6082HVG/latent_factors.csv')

In [ ]:
weights = m_more.get_weights(df=True)


In [ ]:
weights.sort_values(by='Factor1', ascending=False).head(10)

In [ ]:
weights.to_csv('../MOFA/MOFA_15_Factors_6082HVG/weights.csv')

In [ ]:
# latent_factors = latent_factors.reindex(adata.obs_names)

In [ ]:
# adata_combined.obs = adata_combined.obs.join(latent_factors)

In [ ]:
plt.rcParams['figure.figsize'] = (10,5)
mfx.plot_factors(m_more, x='Factor1', y='Factor3', alpha=.1,
                 color="Level_1", group_label='Dataset')

In [ ]:
# m_more.run_umap()

In [ ]:
samples_metadata = m_more.samples_metadata

In [ ]:
# plt.figure(figsize=(10, 8))
# sns.scatterplot(x='UMAP1', y='UMAP2', hue='Level_1', data=samples_metadata, palette='Spectral', s=10)
# plt.title('UMAP Projection with Clusters')
# plt.xlabel('UMAP1')
# plt.ylabel('UMAP2')
# plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
# plt.show()
#TODO Remove?

In [ ]:
# df_randomized = samples_metadata.sample(frac=1, random_state=42).reset_index(drop=True)  
#TODO Remove?

In [ ]:
# plt.figure(figsize=(10, 8))
# sns.scatterplot(x='UMAP1', y='UMAP2', hue='Condition', data=df_randomized, palette='Spectral', s=10)
# plt.title('UMAP Projection with Clusters')
# plt.xlabel('UMAP1')
# plt.ylabel('UMAP2')
# plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
# plt.show()
#TODO Remove?

In [ ]:
samples_metadata = samples_metadata.join(latent_factors)

In [ ]:
# plt.figure(figsize=(10, 8))
# sns.scatterplot(x='UMAP1', y='UMAP2', hue='Factor1', data=samples_metadata, palette='Spectral', s=10)
# plt.title('UMAP Projection with Clusters')
# plt.xlabel('UMAP1')
# plt.ylabel('UMAP2')
# plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
# plt.show()
#TODO Remove?

In [ ]:
# Check if mfx.plot_weights() supports a figsize argument
plt.rcParams['figure.figsize'] = (20, 30)
ax_plot = mfx.plot_weights(m_more, factors=range(0, 15), n_features=10,
                           y_repel_coef=0.5, x_rank_offset=-150)

# Adjust the font size of the gene labels
for text in ax_plot.texts:
    text.set_fontsize(10)

plt.show()

In [ ]:
mfx.plot_weights_scatter(m_more, x="Factor0", y="Factor1",
                         hist=True, n_features=20)

In [ ]:
mfx.plot_weights_heatmap(m_more, n_features=5, 
                         factors=range(0, 15), 
                         xticklabels_size=6, w_abs=True, 
                         cmap="viridis", cluster_factors=False)

In [ ]:
mfx.plot_weights_heatmap(m_more, n_features=10, 
                         factors=range(0, 10), 
                         xticklabels_size=6, w_abs=True, 
                         cmap="viridis", cluster_factors=False)

In [ ]:
mfx.plot_factors_correlation(m_more)

In [ ]:
mfx.plot_weights_correlation(m_more)


In [ ]:
# m.get_r2(factors=list(range(20))).sort_values("R2", ascending=False)

In [ ]:
plt.rcParams['figure.figsize'] = (10,5)
mfx.plot_r2(m_more, factors=list(range(15)), cmap="Blues" )

In [ ]:
r2_celltypes = m_more.get_r2(group_label='Level_1', factors=list(range(10)))

In [ ]:
dict_map ={
    'NK Cell': 'Natural Killer',
    'Ductal Cell/Malignant' : 'Malignant',
    'Intra-pancreatic Neurons': 'Neuronal Cell'
}
m_more.metadata.Level_1 = m_more.metadata.Level_1.replace('NK Cell', 'Natural Killer')
m_more.metadata.Level_1 = m_more.metadata.Level_1.replace('Ductal Cell/Malignant', 'Malignant')
m_more.metadata.Level_1 = m_more.metadata.Level_1.replace('Intra-pancreatic Neurons', 'Neuronal Cell')


In [ ]:
pwd

In [ ]:
plt.rcParams['figure.figsize'] = (20,20)
mfx.plot_factors_scatter(m_more, color='Level_1')


In [ ]:
r2_celltypes = m_more.calculate_variance_explained(factors=list(range(15)), group_label='Level_1', per_factor=True, )

In [ ]:
pivot_df = r2_celltypes.pivot(index='Factor', columns='Group', values='R2')
plt.figure(figsize=(10, 5)) 
sns.heatmap(pivot_df, cmap='Blues', annot=False)
plt.title('R² Heatmap by Factor and Group')
plt.xlabel('Group')
plt.ylabel('Factor')
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))  
sns.clustermap(pivot_df.T, cmap='Blues', standard_scale=1, annot=False)
plt.yticks(rotation=0)
plt.show()

In [ ]:
cols = ['Factor' + str(i + 1) for i in list(range(10))]

In [ ]:
plt.figure(figsize=(10, 5))  
sns.clustermap(pivot_df.T[cols], cmap='Blues', standard_scale=1, annot=False)
plt.yticks(rotation=0)
plt.show()

## filtered data (removed amb cells and outliers) with 15 factors and 6082 HVG(120000 HVG and Intersection) - integration done with batch covariate Donor

### the goal of now is to look at the 15 factors and make them more specific to biological processes or cell types by removing overlapping genes

## Analysis of Factors

In [ ]:
weights = pd.read_csv('../MOFA/MOFA_15_Factors_6082HVG/weights.csv', index_col='Unnamed: 0')
genes = pd.read_csv('../MOFA/MOFA_15_Factors_6082HVG/genes.csv', index_col='Unnamed: 0')
factors = pd.read_csv('../MOFA/MOFA_15_Factors_6082HVG/latent_factors.csv', index_col='Unnamed: 0')

In [ ]:
from collections import Counter

In [ ]:
len((genes[:400].values.flatten()))

In [ ]:
count_df = pd.DataFrame.from_dict(Counter(genes[:400].values.flatten()), orient='index').sort_values(by=0, ascending=False)

In [ ]:
count_df.head() #[count_df[0] == 15]

In [ ]:
genes.shape

In [ ]:
def remove_overlapping_genes(column):
    global assigned_genes
    # Create a new column with the same length, fill it with NaN initially
    unique_genes = [np.nan] * len(column)
    # Iterate over the column, keep only genes that are not assigned
    for i, gene in enumerate(column):
        if gene not in assigned_genes:
            unique_genes[i] = gene
            assigned_genes.add(gene)
    return unique_genes


In [ ]:
top_500_per_factor = genes[:500]

In [ ]:
len(set(top_500_per_factor.values.flatten()))

In [ ]:
assigned_genes = set()
for col in top_500_per_factor.columns:
    top_500_per_factor[col] = remove_overlapping_genes(top_500_per_factor[col])

In [ ]:
top_500_per_factor = pd.DataFrame({col: top_500_per_factor[col].dropna().reset_index(drop=True) for col in top_500_per_factor.columns}).reindex(range(len(top_500_per_factor)))

In [ ]:
dict_gp = top_400_per_factor.to_dict(orient='list')
#TODO top_400_per_factor is not defined

In [ ]:
import math
def is_not_nan(value):
    return not (isinstance(value, float) and math.isnan(value))
cleaned_dict_gp = {key: [gene for gene in genes if is_not_nan(gene)] for key, genes in dict_gp.items()}

In [ ]:
for k,v in cleaned_dict_gp.items():
    # print(k)
    name = k.replace(' ','_') + '_score'
    print(f'Computing scorea for {k}')
    sc.tl.score_genes(adata, v, score_name=name, layer='log_norm', ctrl_as_ref=False)

In [ ]:
def make_score_cols(name=None):
    temp_list = []
    for i in adata.obs.columns:
        if 'Factor' in i:
            temp_list.append(i)
    if name:
        temp_list.append(name)
    return temp_list

score_cols = make_score_cols()

sc.pp.pca(adata)
sc.tl.dendrogram(adata, groupby='batch_covariate')
sc.pl.matrixplot(adata, score_cols, groupby='batch_covariate',  dendrogram=True, standard_scale='var', figsize=(10, 10), swap_axes=False)   

In [ ]:
adata.obs.Level_1 = adata.obs.Level_1.replace('NK Cell', 'Natural Killer')
adata.obs.Level_1 = adata.obs.Level_1.replace('Ductal Cell/Malignant', 'Malignant')
adata.obs.Level_1 = adata.obs.Level_1.replace('Intra-pancreatic Neurons', 'Neuronal Cell')

In [ ]:
sc.tl.dendrogram(adata, groupby='Level_1')
sc.pl.matrixplot(adata, score_cols[:10], groupby='Level_1',  dendrogram=True, figsize=(10, 10), swap_axes=False, standard_scale='var')   

In [ ]:
plt.rcParams['figure.figsize'] = (15,5)
for fac in score_cols:
    fig, ax = plt.subplots()  
    sc.pl.violin(adata, keys=fac, groupby='Level_1', ax=ax, show=False)
    ax.set_xticklabels(ax.get_xticklabels(), fontsize=8, rotation=45, ha='right', rotation_mode='anchor') 
    plt.show()  

In [ ]:
all_genes = [x for x in list(set(top_500_per_factor.values.flatten().astype(str))) if x!= 'nan']
binary_matrix = pd.DataFrame(0, index=all_genes, columns=top_500_per_factor.columns)
for column in top_500_per_factor.columns:
    genes_in_program = top_500_per_factor[column].dropna()
    binary_matrix.loc[genes_in_program, column] = 1

In [ ]:
binary_matrix.to_csv('../MOFA/MOFA_15_Factors_6082HVG/binary_matrix_15_factors.csv')

# TOP 10 factors selected

In [ ]:
# select first 10 factors
cols = ['Factor' + str(i+1) for i in list(range(10))]
subset = top_500_per_factor[cols]

In [ ]:
all_genes = [x for x in list(set(subset.values.flatten().astype(str))) if x!= 'nan']
binary_matrix = pd.DataFrame(0, index=all_genes, columns=subset.columns)
for column in subset.columns:
    genes_in_program = subset[column].dropna()
    binary_matrix.loc[genes_in_program, column] = 1

In [ ]:
binary_matrix.to_csv('../MOFA/MOFA_15_Factors_6082HVG/binary_matrix.csv')

# GSEA on 15 factors

In [ ]:
factor_gene_dict = {}

# Ensure you're selecting the first 500 rows of the DataFrame first and the first 10 columns
df_subset = binary_matrix

# Iterate over each factor (column) to collect genes where value is 1
for factor in df_subset.columns:
    # Collect genes where the value is 1 for the given factor (assuming values are numeric)
    genes = df_subset.index[df_subset[factor] == 1].tolist()
    factor_gene_dict[factor] = genes

# Step 2: Print the number of genes for each factor
number_genes = []
for factor, genes in factor_gene_dict.items():
    print(f"Factor: {factor}, Number of genes: {len(genes)}")
    number_genes.append(len(genes))
print(f'Total Genes: {sum(number_genes)}')

In [ ]:
len(factor_gene_dict.values())

In [ ]:
results = {}
# Perform GSEA per factor
for factor, genes in factor_gene_dict.items():
    print(f"Performing GSEA for {factor}...")

    # Perform enrichment analysis for the current factor
    gsea_results = gp.enrichr(
        gene_list=genes,   # Single factor gene set
        organism='Human',  # Set the organism, e.g., 'Human'
        gene_sets='KEGG_2021_Human',  # Description for the output
        cutoff=0.05  # p-value threshold (optional)
    )
    sorted_df = gsea_results.res2d.sort_values(by=['Adjusted P-value', 'Combined Score'], ascending=[True, False])
    results[f'GSEA_{factor}'] = sorted_df 
    print(f'Gene Sets Enriched for {factor}: {sorted_df.head(10).Term.tolist()}')
    # Output results for the current factor
    print(f"GSEA results for {factor}:")
    print("\n")

In [ ]:
results['GSEA_Factor1'].head(5)

In [ ]:
import pickle
with open('gsea_MOFA_15_factors.pkl', 'wb') as f:
    pickle.dump(results, f)